In [ ]:
from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig
from asgardpy.data.target import apply_selection_mask_to_models

from gammapy.modeling.models import Models, PowerLawSpectralModel

import matplotlib.pyplot as plt
import numpy as np

import astropy.units as u
import logging

In [ ]:
from asgardpy.data.dataset_3d import Dataset3DGeneration

In [ ]:
log = logging.getLogger("test 3d dataset")

In [ ]:
config_file = "/path/to/Config.yaml"

In [ ]:
config = AsgardpyConfig()

In [ ]:
config

In [ ]:
%%time
config_main = config.read(config_file)

In [ ]:
for g in config_main.general:
    print(g)

# Steps mentioned in the main config file

In [ ]:
config_main.general.steps

# Target source information

In [ ]:
for c in config_main.target:
    print(c)

# 3D Datasets informations

In [ ]:
for cc in config_main.dataset3d:
    print(cc)

In [ ]:
for cc in config_main.fit_params:
    print(cc)

In [ ]:
config_main.general.log.dict()

In [ ]:
%%time
analysis = AsgardpyAnalysis(config_main)

In [ ]:
analysis

# Prepare for a single 3D Dataset, based on instrument and key_name

In [ ]:
instruments_list = config_main.dataset3d.instruments
print(len(instruments_list), "number of 3D dataset information provided")

config_3d_dataset = instruments_list[0]
print(f"Instrument selected is {config_3d_dataset.name}")

In [ ]:
key_names = config_3d_dataset.dataset_info.key
print(f"The list of different keys or modes of observations for the selected instrument are {key_names}")

# Run each Dataset3DGeneration step

In [ ]:
%%time
generate_3d_dataset = Dataset3DGeneration(
    log, config_3d_dataset, config_main
)

## Initialized variables

In [ ]:
print(generate_3d_dataset.config_target)

In [ ]:
print(generate_3d_dataset.exclusion_mask)

In [ ]:
print(generate_3d_dataset.irfs)

In [ ]:
print(generate_3d_dataset.events)

In [ ]:
print(generate_3d_dataset.diffuse_models)

In [ ]:
print(generate_3d_dataset.list_sources)

### First check for the given file list if they are readable or not.

In [ ]:
%%time
file_list = generate_3d_dataset.read_to_objects(key_names[0])

In [ ]:
print(file_list)

In [ ]:
for k in file_list.keys():
    print(k, file_list[k])

In [ ]:
print(generate_3d_dataset.irfs["exposure"])

In [ ]:
print(generate_3d_dataset.irfs["psf"])

In [ ]:
print(generate_3d_dataset.irfs["edisp"].info())

In [ ]:
print(generate_3d_dataset.diffuse_models["gal_diffuse"])

In [ ]:
print(generate_3d_dataset.diffuse_models["iso_diffuse"])

In [ ]:
print(len(generate_3d_dataset.list_sources), "number of sources read from XML file")

In [ ]:
for src in generate_3d_dataset.list_sources:
    print(src)

In [ ]:
print("Number of free parameters in the whole list of SkyModels:", len(Models(generate_3d_dataset.list_sources).parameters.free_parameters))

In [ ]:
print("Names of the sources in the whole list of SkyModels:", Models(generate_3d_dataset.list_sources).names)

### Start preparing objects to create the counts map

In [ ]:
%%time
generate_3d_dataset.set_energy_dispersion_matrix()

In [ ]:
print(generate_3d_dataset.irfs["edisp_kernel"])
generate_3d_dataset.irfs["edisp_kernel"].peek()

In [ ]:
%%time
generate_3d_dataset.load_events(file_list["events_file"])

### EventList and GTI

In [ ]:
generate_3d_dataset.events["events"].peek()

In [ ]:
print(generate_3d_dataset.events["gti"])

In [ ]:
%%time
# Source position from the Events file
source_pos, evts_radius = generate_3d_dataset.get_source_skycoord()
print(source_pos, evts_radius)

## Create the Counts Map

In [ ]:
%%time
generate_3d_dataset.create_counts_map(source_pos, evts_radius)

In [ ]:
generate_3d_dataset.events["counts_map"]

In [ ]:
generate_3d_dataset.events["counts_map"].plot_interactive()

In [ ]:
generate_3d_dataset.events["counts_map"].sum_over_axes().smooth(2).plot(stretch="sqrt", add_cbar=True)

### Exclusion mask

In [ ]:
%%time
generate_3d_dataset.create_exclusion_mask()

In [ ]:
print(generate_3d_dataset.exclusion_mask)

In [ ]:
generate_3d_dataset.exclusion_mask.plot_interactive()

### IRF interpolators

In [ ]:
%%time
generate_3d_dataset.set_edisp_interpolator()

In [ ]:
print(generate_3d_dataset.irfs["edisp_interp_kernel"])
generate_3d_dataset.irfs["edisp_interp_kernel"].peek()

In [ ]:
%%time
generate_3d_dataset.set_exposure_interpolator()

In [ ]:
print(generate_3d_dataset.irfs["exposure_interp"])

### Diffuse background cutout

In [ ]:
%%time
generate_3d_dataset.generate_diffuse_background_cutout()

In [ ]:
print(Models(generate_3d_dataset.list_sources)["diffuse-iem"])

In [ ]:
print(generate_3d_dataset.diffuse_models["gal_diffuse_cutout"])

# Apply Exclusion mask to models

In [ ]:
generate_3d_dataset.list_sources = apply_selection_mask_to_models(
    generate_3d_dataset.list_sources,
    target_source=generate_3d_dataset.config_target.source_name,
    selection_mask=generate_3d_dataset.exclusion_mask,
)

In [ ]:
for m in generate_3d_dataset.list_sources:
    print(m)

### Produce MapDataset

In [ ]:
%%time
dataset = generate_3d_dataset.generate_dataset(key_names[0])

In [ ]:
print(dataset)

In [ ]:
for m in generate_3d_dataset.list_sources:
    print(m)

# Assigning models to the dataset

In [ ]:
%%time
dataset.models = generate_3d_dataset.list_sources

In [ ]:
%%time
print(dataset)

# Some basic plots from the Dataset

In [ ]:
dataset.mask_safe.plot_interactive()

In [ ]:
fig = plt.figure(figsize=(6, 3.8), dpi=120)
vmin, vmax = np.percentile(
    dataset.counts.sum_over_axes().smooth("0.1 deg").data, 
    [5, 99.9]
)
ax = dataset.counts.sum_over_axes().smooth("0.2 deg").plot(
    vmin=vmin, vmax=vmax, add_cbar=True, stretch="sqrt",
)
ax.set_title('CountsMap')

In [ ]:
dataset.plot_residuals_spectral()

In [ ]:
dataset.plot_residuals_spatial()

In [ ]:
dataset.info_dict()

In [ ]:
plt.figure(figsize=(8, 5))
axs = plt.subplot(111, projection=dataset.counts.geom.wcs)

dataset.counts.sum_over_axes().smooth(0.05 * u.deg).plot(
    ax=axs, stretch="sqrt", add_cbar=True, cmap="afmhot"
)
dataset.models.plot_regions(ax=axs, color="white")
axs.set_title(dataset.name)

plt.show()

In [ ]:
dataset.exposure.plot_interactive(add_cbar=True)

In [ ]:
dataset.psf.plot_containment_radius_vs_energy()

In [ ]:
plt.figure(figsize=(8, 5))

energy = [100, 300, 1000] * u.GeV
dataset.psf.plot_psf_vs_rad(energy_true=energy)

spectrum = PowerLawSpectralModel(index=1.25)
psf_mean = dataset.psf.to_image(spectrum=spectrum)
psf_mean.plot_psf_vs_rad(c="k", ls="--", energy_true=[500] * u.GeV)

plt.xlim(1e-3, 0.3)
plt.ylim(1e3, 1e6)
plt.legend()

In [ ]:
radius = dataset.psf.containment_radius(energy_true=1 * u.TeV, fraction=0.95)
print(radius)

In [ ]:
plt.figure()
edisp_kernel = dataset.edisp.get_edisp_kernel()
edisp_kernel.plot_matrix()

In [ ]:
plt.figure()
npred = dataset.npred()
npred.sum_over_axes().plot(add_cbar=True)